<a href="https://colab.research.google.com/github/AndrePaind/ERA5-Last10-Days-Rainfall/blob/main/ERA5_Last10_Days_Rainfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# ERA5 Daily Rainfall (mm/day) — LAST 10 DAYS
# Robust to mixed lon conventions and tiny lat edge differences.
#
# Input: upload "All Locations Colombia.csv" via Colab file picker
# Output:
#   - CSV:  /content/era5_municipalities_points_daily_rain_mm_last10days_until_YYYYMMDD.csv
#   - Google Sheet: ERA5_Rainfall_last10days_until_YYYYMMDD
#       * Sheet "data": raw table
#       * Sheet "pivot": municipalities x days, SUM(tp_mm)
#
# Columns in "data": date, Codigo_DA, DANE, department, municipality, lat, lon, tp_mm
# ============================================

# ---- SETTINGS ----
PAD_DEG              = 1.0
SPREADSHEET_BASENAME = "ERA5_Rainfall"

# ---- INSTALLS ----
!pip -q install -U cdsapi xarray netCDF4 pandas tqdm gspread gspread_dataframe

# ---- IMPORTS ----
import os, re
from datetime import date, timedelta

import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

# ---------- UPLOAD LOCATIONS CSV ----------
from google.colab import files

print("📁 Please upload your locations file (e.g. 'All Locations Colombia.csv')...")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded. Please upload your locations CSV.")

CSV_PATH = list(uploaded.keys())[0]
print(f"✅ Using locations file: {CSV_PATH}")

# ---------- READ & CLEAN CSV ----------
def _norm(s):
    return (s.strip().lower()
            .replace("ó","o").replace("á","a").replace("é","e").replace("í","i").replace("ú","u")
            .replace("ï","i").replace("ö","o").replace("ü","u"))

read_ok, df_raw, last_err = False, None, None
for enc in ("utf-8","utf-8-sig","latin-1"):
    for sep in [",",";","\t","|"]:
        try:
            tmp = pd.read_csv(CSV_PATH, sep=sep, encoding=enc)
            cols_low = [_norm(c) for c in tmp.columns]
            ok = (
                any("municipio" in c for c in cols_low) and
                any(("departamen" in c) or ("departamento" in c) or ("depto" in c) for c in cols_low) and
                any(c in {"codigo da","codigo_da","codigo da.","código da","codigo_da."} for c in cols_low) and
                any(c in {"dane","codigo dane","codigo_dane","código dane"} for c in cols_low) and
                any(c in {"new_lat","lat","latitude"} for c in cols_low) and
                any(c in {"new_lon","lon","longitude"} for c in cols_low)
            )
            if ok:
                df_raw = tmp.copy(); read_ok = True; break
        except Exception as e:
            last_err = e
    if read_ok: break
if not read_ok:
    raise ValueError(f"Cannot read '{CSV_PATH}'. Last error: {last_err}")

df_norm = df_raw.copy()
df_norm.columns = [_norm(c) for c in df_norm.columns]

def pick(opts):
    for o in opts:
        if o in df_norm.columns: return o
    return None

c_codigo_da = pick(["codigo da","codigo_da","codigo da.","código da","codigo_da."])
c_dane      = pick(["dane","codigo dane","codigo_dane","código dane"])
c_muni      = pick(["municipio"])
c_dept      = pick(["departamen","departamento","depto"])
c_lat       = pick(["new_lat","lat","latitude"])
c_lon       = pick(["new_lon","lon","longitude"])
if None in [c_codigo_da,c_dane,c_muni,c_dept,c_lat,c_lon]:
    raise ValueError(f"Missing required columns. Found: {list(df_norm.columns)}")

df = df_norm[[c_codigo_da,c_dane,c_dept,c_muni,c_lat,c_lon]].copy()
df.columns = ["Codigo_DA","DANE","department","municipality","lat_raw","lon_raw"]

def clean_coord(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("\u00A0"," ").replace(" ","")
    s = s.replace("°","").replace("'","").replace("’","").replace("”","").replace("“","").replace("\"","")
    s = s.replace(",", ".")
    m = re.match(r"^-?\d+(\.\d+)?", s)
    return float(m.group(0)) if m else np.nan

df["lat"] = df["lat_raw"].apply(clean_coord)
df["lon"] = df["lon_raw"].apply(clean_coord)

# fix swapped lat/lon if needed
swap_mask = (df["lat"].abs() > 90) & (df["lon"].abs() <= 90)
df.loc[swap_mask, ["lat","lon"]] = df.loc[swap_mask, ["lon","lat"]].values

df["Codigo_DA"] = df["Codigo_DA"].astype(str).str.strip()
df["DANE"]      = df["DANE"].astype(str).str.strip()
df = df.dropna(subset=["lat","lon"])
df = df[df["lat"].between(-90,90) & df["lon"].between(-180,180)].copy()
df = df.drop_duplicates(subset=["Codigo_DA","DANE","lat","lon"]).reset_index(drop=True)

# ---------- BBOX ----------
PAD = PAD_DEG
AREA = [float(df["lat"].max()+PAD), float(df["lon"].min()-PAD),
        float(df["lat"].min()-PAD), float(df["lon"].max()+PAD)]  # [N,W,S,E]
pts = df[["Codigo_DA","DANE","department","municipality","lat","lon"]].copy()
print(f"✅ Loaded {len(pts)} points. BBOX [N,W,S,E]: {AREA}")

# ---------- CDS TOKEN ----------
from google.colab import userdata
TOKEN = userdata.get('CopernicusATRAM')
if not TOKEN:
    raise RuntimeError("Colab secret 'CopernicusATRAM' not found.")
os.makedirs(os.path.expanduser("~"), exist_ok=True)
with open(os.path.expanduser("~/.cdsapirc"), "w") as f:
    f.write("url: https://cds.climate.copernicus.eu/api\n")
    f.write(f"key: {TOKEN}\n")
print("✅ CDS API configured with key 'CopernicusATRAM'")

# ---------- GOOGLE AUTH ----------
from google.colab import auth
auth.authenticate_user()
import google.auth, gspread
from gspread_dataframe import set_with_dataframe
creds, _ = google.auth.default()
gc = gspread.authorize(creds)
print("✅ Google Drive / Sheets auth OK")

# ---------- ERA5 HELPERS ----------
import cdsapi
c = cdsapi.Client()
BASE_OUTDIR = "/content/era5_bbox_daily_tp_last10days"
os.makedirs(BASE_OUTDIR, exist_ok=True)

def retrieve_month(y, m, days, target, area):
    """
    Download ONLY the requested 'days' of that (year,month),
    not the whole month.
    """
    c.retrieve(
        "derived-era5-single-levels-daily-statistics",
        {
            "product_type": "reanalysis",
            "variable": ["total_precipitation"],
            "daily_statistic": "daily_sum",
            "frequency": "1_hourly",
            "time_zone": "utc+00:00",
            "year": str(y),
            "month": f"{m:02d}",
            "day": [f"{d:02d}" for d in days],
            "area": area,        # [N, W, S, E]
            "format": "netcdf",
        },
        target
    )

def _standardize_lonlat(ds: xr.Dataset) -> xr.Dataset:
    # ensure coordinate names
    if "longitude" not in ds.coords and "lon" in ds.coords:
        ds = ds.rename({"lon":"longitude"})
    if "latitude"  not in ds.coords and "lat" in ds.coords:
        ds = ds.rename({"lat":"latitude"})
    # longitude to [-180,180)
    if "longitude" in ds.coords:
        lon = ds["longitude"]
        lon_std = ((lon + 180.0) % 360.0) - 180.0
        ds = ds.assign_coords(longitude=lon_std).sortby("longitude")
        # drop dupes after wrap
        lon_vals = np.round(ds["longitude"].values, 6)
        _, idx = np.unique(lon_vals, return_index=True)
        if len(idx) != ds.sizes["longitude"]:
            ds = ds.isel(longitude=np.sort(idx))
    # latitude sorted descending (ERA5 convention)
    if "latitude" in ds.coords:
        ds = ds.sortby("latitude", ascending=False)
    return ds

def _preprocess_month(ds: xr.Dataset) -> xr.Dataset:
    ds = _standardize_lonlat(ds)
    if "valid_time" in ds.coords and "time" not in ds.coords:
        ds = ds.rename({"valid_time":"time"})
    if "number" in ds.dims:
        ds = ds.isel(number=0)
    keep = [v for v in ds.data_vars if v in ("tp","total_precipitation")]
    ds = ds[keep].sortby("time")
    # round coords to reduce fp jitter
    if "latitude" in ds.coords:
        ds = ds.assign_coords(latitude=np.round(ds["latitude"].values, 5))
    if "longitude" in ds.coords:
        ds = ds.assign_coords(longitude=np.round(ds["longitude"].values, 5))
    return ds

def open_tp(files):
    """Open all month files on a common grid and return tp_mm (mm/day)."""
    ds0 = _preprocess_month(xr.open_dataset(files[0]))
    lat_template = ds0["latitude"].values
    lon_template = ds0["longitude"].values

    dsets = [ds0]
    for f in files[1:]:
        dsi = _preprocess_month(xr.open_dataset(f))
        dsi = dsi.reindex(latitude=lat_template, longitude=lon_template,
                          method="nearest", tolerance=1e-3)
        dsets.append(dsi)

    ds = xr.concat(dsets, dim="time", data_vars="minimal", coords="minimal",
                   compat="override", join="override")
    var = "tp" if "tp" in ds.data_vars else "total_precipitation"
    da = ds[var]
    # m/day → mm/day
    return (da * 1000.0).rename("tp_mm")

def norm_lon_factory(tp_da):
    # grid already standardized to [-180,180)
    return lambda lon: ((lon + 180.0) % 360.0) - 180.0

# ---------- LAST 10 DAYS WINDOW ----------
# ERA5 is typically complete up to yesterday → we stop at yesterday.
end_date = date.today() - timedelta(days=1)
start_date = end_date - timedelta(days=9)
print(f"📅 Using window from {start_date} to {end_date} (10 days)")

# List of individual dates
last10 = []
cur = start_date
while cur <= end_date:
    last10.append(cur)
    cur += timedelta(days=1)

# Unique (year, month) pairs in that 10-day window
ym_set = sorted({(d.year, d.month) for d in last10})
print("🗓️ Year-month combinations:", ym_set)

def days_for_month_in_window(y, m, dates):
    return sorted({d.day for d in dates if d.year == y and d.month == m})

# ---------- DOWNLOAD ONLY THE NEEDED DAYS ----------
files = []
for (yy, mm) in ym_set:
    days = days_for_month_in_window(yy, mm, last10)
    outdir = os.path.join(BASE_OUTDIR, f"{yy}")
    os.makedirs(outdir, exist_ok=True)
    tgt = f"{outdir}/era5_tp_daily_sum_{yy}_{mm:02d}.nc"
    if not os.path.exists(tgt):
        print(f"⬇️ Downloading ERA5 daily stats for {yy}-{mm:02d} days {days}")
        retrieve_month(yy, mm, days, tgt, AREA)
    else:
        print(f"✅ Already present: {yy}-{mm:02d} (file reused)")
    files.append(tgt)

# ---------- OPEN + FILTER TO LAST 10 DAYS (SAFETY) ----------
tp = open_tp(files)
norm_lon = norm_lon_factory(tp)

tp_10 = tp.sel(time=slice(str(start_date), str(end_date)))
print("🕒 Resulting time steps:", tp_10["time"].values)

# ---------- EXTRACT SERIES FOR EACH POINT ----------
rows = []
for _, r in tqdm(pts.iterrows(), total=len(pts), desc="Extracting last 10 days"):
    series = tp_10.sel(
        latitude=float(r["lat"]),
        longitude=norm_lon(float(r["lon"])),
        method="nearest"
    )
    # ✅ enforce DAILY TOTALS (mm/day)
    series_daily = series.resample(time="1D").sum()

    df_pt = series_daily.to_dataframe().reset_index()[["time","tp_mm"]]
    df_pt.insert(0, "Codigo_DA", r["Codigo_DA"])
    df_pt.insert(1, "DANE", r["DANE"])
    df_pt.insert(2, "department", r["department"])
    df_pt.insert(3, "municipality", r["municipality"])
    df_pt["lat"] = float(r["lat"])
    df_pt["lon"] = float(r["lon"])
    rows.append(df_pt)

all_df = pd.concat(rows, ignore_index=True).rename(columns={"time":"date"})
all_df = all_df[["date","Codigo_DA","DANE","department","municipality","lat","lon","tp_mm"]]
all_df = all_df.sort_values(["Codigo_DA","date"]).reset_index(drop=True)

# ---------- SAVE CSV ----------
csv_end_str = end_date.strftime("%Y%m%d")
out_csv = f"/content/era5_municipalities_points_daily_rain_mm_last10days_until_{csv_end_str}.csv"
all_df.to_csv(out_csv, index=False)
print(f"✅ CSV saved: {out_csv}  ({len(all_df)} rows)")

# ---------- GOOGLE SHEET (RAW DATA + PIVOT) ----------
sheet_name = f"{SPREADSHEET_BASENAME}_last10days_until_{csv_end_str}"

try:
    sh = gc.open(sheet_name)
    try:
        ws_old = sh.worksheet("data"); sh.del_worksheet(ws_old)
    except gspread.WorksheetNotFound:
        pass
except gspread.SpreadsheetNotFound:
    sh = gc.create(sheet_name)
    print(f"📄 Created spreadsheet '{sheet_name}' in your Drive.")

# "data" sheet with flat table
ws_data = sh.add_worksheet(title="data", rows="1000", cols="12")
set_with_dataframe(ws_data, all_df, include_index=False, resize=True)
print(f"✅ Uploaded raw data to Google Sheets → {sheet_name} / data")

# Create / recreate "pivot" sheet
try:
    ws_pivot_old = sh.worksheet("pivot")
    sh.del_worksheet(ws_pivot_old)
except gspread.WorksheetNotFound:
    pass
ws_pivot = sh.add_worksheet(title="pivot", rows="1000", cols="26")

# Build pivot table:
# - rows: municipality (column index 4)
# - columns: date (column index 0)
# - values: SUM of tp_mm (column index 7)
nrows = len(all_df) + 1  # + header row
data_sheet_id = ws_data.id
pivot_sheet_id = ws_pivot.id

pivot_body = {
    "requests": [
        {
            "updateCells": {
                "start": {
                    "sheetId": pivot_sheet_id,
                    "rowIndex": 0,
                    "columnIndex": 0,
                },
                "rows": [
                    {
                        "values": [
                            {
                                "pivotTable": {
                                    "source": {
                                        "sheetId": data_sheet_id,
                                        "startRowIndex": 0,
                                        "startColumnIndex": 0,
                                        "endRowIndex": nrows,
                                        "endColumnIndex": 8,  # A:H
                                    },
                                    "rows": [
                                        {
                                            "sourceColumnOffset": 4,  # municipality
                                            "showTotals": True,
                                            "sortOrder": "ASCENDING",
                                        }
                                    ],
                                    "columns": [
                                        {
                                            "sourceColumnOffset": 0,  # date
                                            "showTotals": True,
                                            "sortOrder": "ASCENDING",
                                        }
                                    ],
                                    "values": [
                                        {
                                            "summarizeFunction": "SUM",
                                            "sourceColumnOffset": 7,  # tp_mm
                                            "name": "Total Rain (mm)",
                                        }
                                    ],
                                }
                            }
                        ]
                    }
                ],
                "fields": "pivotTable",
            }
        }
    ]
}

sh.batch_update(pivot_body)
print("✅ Pivot table created → sheet 'pivot' (municipalities x dates, SUM(tp_mm))")

# 🔗 Show Google Sheet URL
print(f"🔗 Google Sheet link: {sh.url}")

print("\n🎉 Done. Last 10 days of DAILY TOTAL rainfall (mm/day) exported to CSV and Google Sheet with pivot.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
📁 Please upload your locations file (e.g. 'All Locations Colombia.csv')...


Saving All Locations Colombia - All Locations Colombia.csv to All Locations Colombia - All Locations Colombia.csv
✅ Using locations file: All Locations Colombia - All Locations Colombia.csv
✅ Loaded 1123 points. BBOX [N,W,S,E]: [14.3540153, -82.705052, -5.203165, -66.0464591]
✅ CDS API configured with key 'CopernicusATRAM'
✅ Google Drive / Sheets auth OK
📅 Using window from 2025-11-04 to 2025-11-13 (10 days)
🗓️ Year-month combinations: [(2025, 11)]
⬇️ Downloading ERA5 daily stats for 2025-11 days [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


2025-11-14 18:16:04,586 INFO Request ID is 8284433c-9961-4d76-9b01-557a2c49aa86
INFO:ecmwf.datastores.legacy_client:Request ID is 8284433c-9961-4d76-9b01-557a2c49aa86
2025-11-14 18:16:04,739 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-14 18:16:26,454 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-11-14 18:16:37,980 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


84edf48f3a2bc69638921c5e243561ec.nc:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

🕒 Resulting time steps: ['2025-11-04T00:00:00.000000000' '2025-11-05T00:00:00.000000000'
 '2025-11-06T00:00:00.000000000' '2025-11-07T00:00:00.000000000'
 '2025-11-08T00:00:00.000000000']


Extracting last 10 days: 100%|██████████| 1123/1123 [00:11<00:00, 94.08it/s]


✅ CSV saved: /content/era5_municipalities_points_daily_rain_mm_last10days_until_20251113.csv  (5615 rows)
📄 Created spreadsheet 'ERA5_Rainfall_last10days_until_20251113' in your Drive.
✅ Uploaded raw data to Google Sheets → ERA5_Rainfall_last10days_until_20251113 / data
✅ Pivot table created → sheet 'pivot' (municipalities x dates, SUM(tp_mm))
🔗 Google Sheet link: https://docs.google.com/spreadsheets/d/1PE0YKTa9Cc0CFF_MeBmTkTWDkBWb3IBNufGrvLNWVyE

🎉 Done. Last 10 days of DAILY TOTAL rainfall (mm/day) exported to CSV and Google Sheet with pivot.
